# Overview

This Notebook will simulate many users trying to talk to the API at the same time.  It's useful for testing out different profiling methods or demonstrating problems like slow database queries and blocking in async.

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import httpx
from typing import Optional


class User:
    def __init__(self, name: str):
        self.name = name
        self.client = httpx.AsyncClient(
            base_url="http://backend:8000",
            event_hooks={
                "request": [self.print_request],
                "response": [self.print_response],
            },
            follow_redirects=True,
        )

    @classmethod
    async def create(cls, name: str, password: str):
        user = User(name)
        await user.login(password)
        return user

    async def print_request(self, request):
        print(f"<{self.name}> {request.method} {request.url}")

    async def print_response(self, resp):
        if resp.is_error:
            print(resp)
            print(resp.text)
            resp.raise_for_status()

    async def login(self, password: str):
        endpoint = "/auth/login"
        data = {"username": self.name, "password": password}
        resp = await self.client.post(endpoint, data=data)
        resp.raise_for_status()
        token = resp.json()["access_token"]
        token_type = resp.json()["token_type"]
        self.client.headers["Authorization"] = f"{token_type} {token}"

    async def me(self):
        endpoint = "/me"
        resp = await self.client.get(endpoint)
        return resp.json()

    async def get_todos(self):
        endpoint = "/todo"
        resp = await self.client.get(endpoint)
        return resp.json()

    async def make_todo(self, title: str, content: str):
        endpoint = "/todo"
        data = {"title": title, "content": content}
        resp = await self.client.post(endpoint, json=data)
        return resp.json()

    async def delete_todo(self, todo_id: str):
        endpoint = f"/todo/{todo_id}"
        resp = await self.client.delete(endpoint)
        return resp.json()

    async def update_todo(
        self, todo_id: str, title: Optional[str], content: Optional[str]
    ):
        endpoint = f"/todo/{todo_id}"
        data = {}
        if title:
            data["title"] = title
        if content:
            data["content"] = content
        resp = await self.client.put(endpoint, json=data)
        return resp.json()

    def __repr__(self):
        return f"<User {self.name}>"

<IPython.core.display.Javascript object>

In [3]:
import asyncio

coros = []

for i in range(1, 11):
    name = f"user{i}"
    coro = User.create(name, "pass")
    coros.append(coro)

users = await asyncio.gather(*coros)
users

<user1> POST http://backend:8000/auth/login
<user2> POST http://backend:8000/auth/login
<user3> POST http://backend:8000/auth/login
<user4> POST http://backend:8000/auth/login
<user5> POST http://backend:8000/auth/login
<user6> POST http://backend:8000/auth/login
<user7> POST http://backend:8000/auth/login
<user8> POST http://backend:8000/auth/login
<user9> POST http://backend:8000/auth/login
<user10> POST http://backend:8000/auth/login


[<User user1>,
 <User user2>,
 <User user3>,
 <User user4>,
 <User user5>,
 <User user6>,
 <User user7>,
 <User user8>,
 <User user9>,
 <User user10>]

<IPython.core.display.Javascript object>

In [4]:
import random

coros = []


async def act(user):
    await asyncio.sleep(random.random())
    await user.me()
    await user.get_todos()
    todo = await user.make_todo(title="note 1", content="content 1")
    await user.update_todo(todo["id"], title="updated title", content="updated content")
    await user.delete_todo(todo["id"])


for user in users:
    coro = act(user)
    coros.append(coro)

coros

[<coroutine object act at 0x7fbfe8073cc0>,
 <coroutine object act at 0x7fbfcf4b75c0>,
 <coroutine object act at 0x7fbfcf4b7640>,
 <coroutine object act at 0x7fbfcf4b76c0>,
 <coroutine object act at 0x7fbfcf4b7540>,
 <coroutine object act at 0x7fbfcf4b7740>,
 <coroutine object act at 0x7fbfcf4b77c0>,
 <coroutine object act at 0x7fbfcf4b7840>,
 <coroutine object act at 0x7fbfcf4b78c0>,
 <coroutine object act at 0x7fbfcf4b79c0>]

<IPython.core.display.Javascript object>

In [5]:
await asyncio.gather(*coros)

<user9> GET http://backend:8000/me
<user9> GET http://backend:8000/todo
<user9> GET http://backend:8000/todo/
<user4> GET http://backend:8000/me
<user4> GET http://backend:8000/todo
<user9> POST http://backend:8000/todo
<user10> GET http://backend:8000/me
<user4> GET http://backend:8000/todo/
<user3> GET http://backend:8000/me
<user9> POST http://backend:8000/todo/
<user10> GET http://backend:8000/todo
<user3> GET http://backend:8000/todo
<user10> GET http://backend:8000/todo/
<user4> POST http://backend:8000/todo
<user3> GET http://backend:8000/todo/
<user4> POST http://backend:8000/todo/
<user2> GET http://backend:8000/me
<user8> GET http://backend:8000/me
<user1> GET http://backend:8000/me
<user2> GET http://backend:8000/todo
<user2> GET http://backend:8000/todo/
<user8> GET http://backend:8000/todo
<user8> GET http://backend:8000/todo/
<user1> GET http://backend:8000/todo
<user1> GET http://backend:8000/todo/
<user7> GET http://backend:8000/me
<user7> GET http://backend:8000/todo
<

[None, None, None, None, None, None, None, None, None, None]

<IPython.core.display.Javascript object>